## A Notebook to get Information from existing IIIF-Manifests, select canvases, and arrange them into a new manifest


NOTE: For now, this will only work with existing manifests following the v2 of the IIIF Presentation API, but will produce a manifest following v3.

In [282]:
import requests
import json

In [283]:
# function to create a new manifest
def create_manifest(manifest_id, label, metadata, summary, provider):
   
    new_manifest = {
        "@context": "http://iiif.io/api/presentation/3/context.json",
        "id": manifest_id,
        "type": "Manifest",
        "label": label,
        "summary": summary,
        "metadata": metadata,
        "provider": provider,
        "items": []  # No canvases yet
    }  

    return new_manifest

In [284]:
# Provide basic Metadata for the new manifest
manifest_id="https://raw.githubusercontent.com/r0man-ist/iiif/refs/heads/main/Sequence_letters.json" ## required
label={"de": ["Hensel, Wilhelm/Fanny Hensel: Briefsequenz aus der Verlobungszeit 1829"]} ## required
metadata=[
    {
         "label": "Autor",
         "value": ["Hensel, Fanny", "Hensel, Wilhelm"]
    },
    {
         "label": "Titel",
         "value": "Briefsequenz aus der Verlobungszeit von Fanny und Wilhelm Hensel, 1829"
    },
    {
      "label": "Signaturen",
      "value": ["MA Depos. 3,6", "MA Depos. 3,4"]
    },  
    {
      "label": "Sprache",
      "value": "ger"
    },
     {
      "label": "Lizenz",
      "value": "Public Domain Mark 1.0"
    },
        ]
summary = { "de": [ "Bei diesem virtuellen Objekt handelt es sich um eine Briefsequenz aus ..." ] }
provider = [
    {
      "id": "https://lab.sbb.berlin",
      "type": "Agent",
      "label": { "en": [ "Stabi Lab" ] },
      "homepage": [
        {
          "id": "https://lab.sbb.berlin",
          "type": "Text",
          "label": { "de": [ "Stabi Lab Homepage" ] },
          "format": "text/html"
        }]
    }      
  ]
    
new_manifest = create_manifest(manifest_id, label, metadata, summary, provider)

In [285]:
# function to get manifests from a list of urls
def fetch_manifests(manifest_urls):
    manifests = []
    for url in manifest_urls:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        manifest = response.json()  # Parse the JSON response
                             
        manifests.append(manifest)
            
    return manifests

In [286]:
# specify a list of manifests to fetch
manifest_urls = ["https://content.staatsbibliothek-berlin.de/dc/1880377578/manifest",
                 "https://content.staatsbibliothek-berlin.de/dc/1878121294/manifest",
                 "https://content.staatsbibliothek-berlin.de/dc/1878144588/manifest"]
manifests = fetch_manifests(manifest_urls)
print(manifests)



[{'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578/manifest', '@type': 'sc:Manifest', '@context': 'http://iiif.io/api/presentation/2/context.json', 'logo': 'https://ngcs-beta.staatsbibliothek-berlin.de/dc/logo-small', 'label': 'Hensel, Wilhelm: Briefe und Billets an Fanny Hensel aus der Verlobungszeit 1829', 'metadata': [{'label': 'Autor', 'value': 'Hensel, Wilhelm'}, {'label': 'Titel', 'value': 'Briefe und Billets an Fanny Hensel aus der Verlobungszeit 1829'}, {'label': 'Projekt', 'value': 'Nachlässe und Autographe digital'}, {'label': '_digitalOrigin', 'value': 'reformatted digital'}, {'label': '_electronicEdition', 'value': '[Electronic ed.]'}, {'label': '_placeOfElectronicOrigin', 'value': 'Berlin'}, {'label': 'Bemerkung', 'value': 'P_SBB_Sondermat_Nachlaesse'}, {'label': 'Bemerkung', 'value': '108 Briefe und Billets'}, {'label': 'BibliographicReference', 'value': '980511'}, {'label': 'Digitalisierer', 'value': 'Staatsbibliothek zu Berlin - Preußischer Kulturbesitz,

In [287]:
# get individual canvases from the manifests
canvases = []
for manifest in manifests:  
    for sequence in manifest.get('sequences', []):
        for canvas in sequence.get('canvases', []):
            if canvas.get('@type') == 'sc:Canvas':
                canvases.append(canvas)

# Print the extracted canvases
print(canvases)

[{'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/canvas', '@type': 'sc:Canvas', 'label': '1r [1]', 'height': 900, 'width': 600, 'images': [{'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/annotation', '@type': 'oa:Annotation', 'motivation': '', 'resource': {'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/full/full/0/default.jpg', '@type': 'dctypes:Image', 'format': '', 'service': {'@context': 'http://iiif.io/api/image/2/context.json', '@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001', 'profile': 'http://iiif.io/api/image/2/level1.json'}}, 'on': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0001/canvas'}]}, {'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0002/canvas', '@type': 'sc:Canvas', 'label': '1v [2]', 'height': 900, 'width': 600, 'images': [{'@id': 'https://content.staatsbibliothek-berlin.de/dc/1880377578-0002/annotation', '@type': 'oa:Annotation', 'motivation': '

In [288]:
# specify a list of canvases to include in the new manifest
# canvases will be included in the order in which they appear in the list
canvas_list = [
    "https://content.staatsbibliothek-berlin.de/dc/1880377578-0243/canvas",
    "https://content.staatsbibliothek-berlin.de/dc/1880377578-0242/canvas",
    "https://content.staatsbibliothek-berlin.de/dc/1878144588-0032/canvas"
]


In [289]:
matching_canvases = []

# Create a dictionary for quick lookup using the canvas IDs
canvas_ids = {canvas["@id"]: canvas for canvas in canvases}

# Iterate over the canvas_list to maintain the specified order
for canvas_id in canvas_list:
    if canvas_id in canvas_ids:
        matching_canvases.append(canvas_ids[canvas_id])

new_manifest["items"] = matching_canvases

In [290]:
# rename keys to comply with V3 of the IIIF Presentation API
for item in new_manifest["items"]:
    if "@type" in item and item["@type"] == "sc:Canvas":
        item["type"] = "canvas"  # Change to "type"
        #del item["@type"]  # Optionally remove the old key

In [291]:

with open("test.json", "w", encoding="utf-8") as f:
        json.dump(new_manifest, f, ensure_ascii=False, indent=4)